# Your Title Here

**Name(s)**: Lauren May and Julia Rehring

**Website Link**: (your website link)

In [3]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!

## Step 1: Introduction

In [89]:
outages = pd.read_excel('outage.xlsx', skiprows=[0,1,2,3,4,6], usecols=lambda x: x not in ['variables', 'OBS'])
outages

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,2011,7.0,Minnesota,MN,...,0.60,91.59,8.41,5.48
1,2014,5.0,Minnesota,MN,...,0.60,91.59,8.41,5.48
2,2010,10.0,Minnesota,MN,...,0.60,91.59,8.41,5.48
...,...,...,...,...,...,...,...,...,...
1531,2009,8.0,South Dakota,SD,...,0.15,98.31,1.69,1.69
1532,2009,8.0,South Dakota,SD,...,0.15,98.31,1.69,1.69
1533,2000,NaN,Alaska,AK,...,0.02,85.76,14.24,2.90


## Step 2: Data Cleaning and Exploratory Data Analysis

In [90]:
# Combine outage start/restoration times and dates into one column

outages['OUTAGE.START.TIME'] = pd.to_timedelta(outages['OUTAGE.START.TIME'].astype(str))
outages['OUTAGE.START'] = outages['OUTAGE.START.DATE'] + outages['OUTAGE.START.TIME']
outages.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME'], inplace=True)

outages['OUTAGE.RESTORATION.TIME'] = pd.to_timedelta(outages['OUTAGE.RESTORATION.TIME'].astype(str))
outages['OUTAGE.RESTORATION'] = outages['OUTAGE.RESTORATION.DATE'] + outages['OUTAGE.RESTORATION.TIME']
outages.drop(columns=['OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], inplace=True)

In [ ]:
# Drop any outages that don't have a month or duration
outages = outages.dropna(subset=['MONTH', 'OUTAGE.DURATION'])

In [7]:
cutoff = 40000
filtered = outages[outages["OUTAGE.DURATION"] <= cutoff]

# Create side-by-side subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Before Outlier Removal", "After Outlier Removal"),
    shared_yaxes=True
)

# Add original distribution
fig.add_trace(
    go.Histogram(
        x=outages["OUTAGE.DURATION"],
        nbinsx=100,
        marker_color='#EF553B',
        name="Original"
    ),
    row=1, col=1
)

# Add filtered distribution
fig.add_trace(
    go.Histogram(
        x=filtered["OUTAGE.DURATION"],
        nbinsx=100,
        marker_color='#00CC96',
        name="Filtered"
    ),
    row=1, col=2
)

# Update layout with axis labels and clean design
fig.update_layout(
    font=dict(family="Arial", size=13),
    bargap=0.05,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='white',
    showlegend=False
)

# Axis labels
fig.update_xaxes(title_text="Outage Duration (minutes)", row=1, col=1)
fig.update_xaxes(title_text="Outage Duration (minutes)", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)

fig.show()


In [9]:
outages = outages[outages['OUTAGE.DURATION'] < 40_000]

In [86]:
print(outages[['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'OUTAGE.START', 'OUTAGE.RESTORATION']].head().to_markdown(index=False))

|   YEAR |   MONTH | U.S._STATE   | POSTAL.CODE   | NERC.REGION   | CLIMATE.REGION     |   ANOMALY.LEVEL | OUTAGE.START        | OUTAGE.RESTORATION   |
|-------:|--------:|:-------------|:--------------|:--------------|:-------------------|----------------:|:--------------------|:---------------------|
|   2011 |       7 | Minnesota    | MN            | MRO           | East North Central |            -0.3 | 2011-07-01 17:00:00 | 2011-07-03 20:00:00  |
|   2014 |       5 | Minnesota    | MN            | MRO           | East North Central |            -0.1 | 2014-05-11 18:38:00 | 2014-05-11 18:39:00  |
|   2010 |      10 | Minnesota    | MN            | MRO           | East North Central |            -1.5 | 2010-10-26 20:00:00 | 2010-10-28 22:00:00  |
|   2012 |       6 | Minnesota    | MN            | MRO           | East North Central |            -0.1 | 2012-06-19 04:30:00 | 2012-06-20 23:00:00  |
|   2015 |       7 | Minnesota    | MN            | MRO           | East North Central |

### Interesting Aggregates

In [12]:
outages['CAUSE.CATEGORY'].value_counts()
outages_grouped = outages.groupby("CAUSE.CATEGORY", as_index=False).agg({
    "OUTAGE.DURATION": "mean",
    "CUSTOMERS.AFFECTED": "sum"
})
print((outages_grouped).to_markdown(index=False))

| CAUSE.CATEGORY                |   OUTAGE.DURATION |   CUSTOMERS.AFFECTED |
|:------------------------------|------------------:|---------------------:|
| equipment failure             |           399.13  |          2.83979e+06 |
| fuel supply emergency         |          8395.23  |          1           |
| intentional attack            |           429.98  |     356315           |
| islanding                     |           200.545 |     209749           |
| public appeal                 |          1468.45  |     159994           |
| severe weather                |          3704.41  |          1.31566e+08 |
| system operability disruption |           728.87  |          1.70555e+07 |


### Univariate Analysis

In [28]:
cause_counts = outages['CAUSE.CATEGORY'].value_counts().reset_index()
cause_counts.columns = ['CAUSE.CATEGORY', 'count']

fig = px.bar(
    cause_counts.sort_values(by='count', ascending=False),
    x='count',
    y='CAUSE.CATEGORY',
    orientation='h',
    title='Distribution of Outage Causes',
    labels={'count': 'Number of Outages', 'CAUSE.CATEGORY': 'Cause'},
    color='count',
    color_continuous_scale='Tealgrn'
)

fig.update_layout(
    coloraxis_showscale=False
)

fig.show()


In [59]:
month_counts = outages['MONTH'].value_counts().reset_index()
month_counts.columns = ['MONTH', 'count']

fig = px.bar(
    month_counts.sort_values(by='count', ascending=False),
    x='MONTH',
    y='count',
    title='Distribution of Months of Outages',
    labels={'count': 'Number of Outages', 'MONTH': 'Month'},
    color_discrete_sequence=['#72B7B2']
)

fig.update_layout(
    coloraxis_showscale=False
)

fig.show()

In [42]:
anomaly_region_counts = outages.groupby(['ANOMALY.LEVEL', 'CLIMATE.CATEGORY']).size().reset_index(name='count')

climate_colors = {
    'warm': 'red',  
    'normal': '#9370DB',   
    'cold': '#1E90FF'     
}

fig2 = px.bar(
    anomaly_region_counts.sort_values(by='count', ascending=False),
    x='ANOMALY.LEVEL',
    y='count',
    color='CLIMATE.CATEGORY',
    color_discrete_map=climate_colors,
    title='Distribution of Anomaly Levels',
    labels={'count': 'Number of Outages', 'ANOMALY.LEVEL': 'Anomaly Level', 'CLIMATE.CATEGORY': 'Climate'},
)

fig2.update_layout(
    legend_title_text='Climate Category'
)

fig2.show()

### Bivariate Analysis

In [26]:
state_avg_duration = outages.groupby('POSTAL.CODE', as_index=False)['OUTAGE.DURATION'].mean()

fig = px.choropleth(
    state_avg_duration,
    locations='POSTAL.CODE',
    locationmode='USA-states',
    color='OUTAGE.DURATION',
    color_continuous_scale='OrRd',
    scope='usa',
    labels={'OUTAGE.DURATION': 'Avg Duration (hrs)'},
    title='Average Outage Duration by State'
)
fig.show()


In [47]:
duration_by_cause = outages.groupby('CAUSE.CATEGORY')['OUTAGE.DURATION'].mean().reset_index()

fig3 = px.bar(
    duration_by_cause.sort_values(by='OUTAGE.DURATION', ascending=False),
    x='OUTAGE.DURATION',
    y='CAUSE.CATEGORY',
    orientation='h',
    title='Average Outage Duration by Cause',
    labels={'OUTAGE.DURATION': 'Avg Duration (hrs)', 'CAUSE.CATEGORY': 'Cause'},
    color='OUTAGE.DURATION',
    color_continuous_scale='Tealgrn'
)

fig3.update_layout(
   coloraxis_showscale=False
)

fig3.show()

In [78]:
duration_by_month = outages.groupby('MONTH')['OUTAGE.DURATION'].mean().reset_index()

fig3 = px.bar(
    duration_by_month.sort_values(by='OUTAGE.DURATION', ascending=False),
    x='MONTH',
    y='OUTAGE.DURATION',
    title='Average Outage Duration by Month',
    labels={'OUTAGE.DURATION': 'Avg Duration (hrs)', 'MONTH': 'Month'},
    color='OUTAGE.DURATION',
    color_continuous_scale='Tealgrn'
)

fig3.update_layout(
   coloraxis_showscale=False
)

fig3.show()

In [ ]:
import statsmodels.api as sm

# Regression line using lowess smoothing
lowess = sm.nonparametric.lowess
smoothed = lowess(filtered_df['DURATION'], filtered_df['ANOMALY_LEVEL'], frac=0.3)

# Create scatter plot
fig = px.scatter(filtered_df, x='ANOMALY_LEVEL', y='DURATION',
                 title='Anomaly Level vs. Outage Duration',
                 labels={'ANOMALY_LEVEL': 'ONI Anomaly Level', 'DURATION': 'Outage Duration (hrs)'})

# Add regression line
fig.add_scatter(x=smoothed[:, 0], y=smoothed[:, 1],
                mode='lines', name='Lowess Trend', line=dict(color='red'))

fig.show()


In [79]:
fig3.write_html('duration_by_month.html', include_plotlyjs='cdn')

## Step 3: Framing a Prediction Problem

In [126]:
# TODO

## Step 4: Baseline Model

In [127]:
outages['CAUSE.CATEGORY'].nunique()

7

In [202]:
# linear regression with cause category (OHE), month (OHE), and climate region (OHE)
from sklearn.pipeline import make_pipeline, FunctionTransformer, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

features = outages[['MONTH', 'CLIMATE.REGION', 'ANOMALY.LEVEL']]
duration = outages['OUTAGE.DURATION']

# make train-test split
X_train, X_test, y_train, y_test = train_test_split(features, duration, random_state=42)

absolute_anomaly = FunctionTransformer(lambda s: np.abs(s))

# OHE ['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY']
preprocessing = make_column_transformer(
    (OneHotEncoder(drop='first'), ['MONTH', 'CLIMATE.REGION']),
    (absolute_anomaly, ['ANOMALY.LEVEL'])
)
model = make_pipeline(
    preprocessing,
    LinearRegression()
)

model.fit(X_train, y_train)

model.predict(pd.DataFrame({
    'MONTH': [7],
    'CLIMATE.REGION': 'Central',
    'ANOMALY.LEVEL': 0.7
}))

array([1538.93])

In [203]:
# training MSE
pt4_mse_train = mean_squared_error(y_train, model.predict(X_train))

# validation MSE
pt4_mse_test = mean_squared_error(y_test, model.predict(X_test))

# validation MAE
pt4_mae_test = mean_absolute_error(y_test, model.predict(X_test))

## Step 5: Final Model

In [207]:
# add grid search for polynomial features, compare Linear, Ridge, and LASSO
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

X_train = outages[['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'POPPCT_URBAN']]
y_train = outages['OUTAGE.DURATION']

# splitting into columns for transformer
poly_column = ['ANOMALY.LEVEL']
other_columns = [col for col in X_train.columns if col not in poly_column]

preprocessor = ColumnTransformer(transformers=[
    ('poly', Pipeline([
        ('poly_features', PolynomialFeatures(include_bias=False))
    ]), poly_column),
    ('other', OneHotEncoder(drop='first'), other_columns)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge()) 
])

param_grid = {
    'preprocessor__poly__poly_features__degree': range(1, 26),
    'regressor__alpha': [0.1, 1.0, 10.0]
}
model_ridge = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
model_ridge.fit(X_train, y_train)

print("Best parameters:", model_ridge.best_params_)
print("Best MSE:", -model_ridge.best_score_)

Best parameters: {'preprocessor__poly__poly_features__degree': 1, 'regressor__alpha': 0.1}
Best MSE: nan


In [208]:
# training MSE
pipe_mse_train = mean_squared_error(y_train, model_ridge.predict(X_train))

# validation MSE
pipe_mse_test = mean_squared_error(y_test, model_ridge.predict(X_test))

# validation MAE
pipe_mae_test = mean_absolute_error(y_test, model_ridge.predict(X_test))

In [209]:
# linear regression with more features
from sklearn.pipeline import make_pipeline, FunctionTransformer, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

features = outages[['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'POPPCT_URBAN']]
duration = outages['OUTAGE.DURATION']

# make train-test split
X_train, X_test, y_train, y_test = train_test_split(features, duration, random_state=42)

absolute_anomaly = FunctionTransformer(lambda s: np.abs(s))

# OHE ['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'CLIMATE.CATEGORY']
preprocessing = make_column_transformer(
    (OneHotEncoder(drop='first'), ['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'CLIMATE.CATEGORY']),
    (StandardScaler(), ['POPPCT_URBAN']),
    (absolute_anomaly, ['ANOMALY.LEVEL'])
)
model_5 = make_pipeline(
    preprocessing,
    LinearRegression()
)

model_5.fit(X_train, y_train)

model_5.predict(pd.DataFrame({
    'MONTH': [7],
    'CLIMATE.REGION': 'Central',
    'CAUSE.CATEGORY': 'severe weather',
    'ANOMALY.LEVEL': 1.1,
    'CLIMATE.CATEGORY': 'cold',
    'POPPCT_URBAN': 70
}))

array([1386.35])

In [210]:
# training MSE
pt5_mse_train = mean_squared_error(y_train, model_5.predict(X_train))

# validation MSE
pt5_mse_test = mean_squared_error(y_test, model_5.predict(X_test))

# validation MAE
pt5_mae_test = mean_absolute_error(y_test, model_5.predict(X_test))

In [211]:
results_data = {
                'Training MSE': [pt4_mse_train, pt5_mse_train, pipe_mse_train], 
                'Testing MSE': [pt4_mse_test, pt5_mse_test, pipe_mse_test], 
                'Testing MAE': [pt4_mae_test, pt5_mae_test, pipe_mae_test]
               }
results_df = pd.DataFrame(results_data, index = ['Part 4', 'Part 5', 'Pipeline'])
results_df

,Training MSE,Testing MSE,Testing MAE
Part 4,1.49e+07,1.27e+07,2404.05
Part 5,1.21e+07,1.00e+07,2166.86
Pipeline,1.07e+07,9.03e+06,1997.37
